# Normal joins

- You've been given two DataFrames to combine into a single useful DataFrame. Your first task is to combine the DataFrames normally and view the execution plan.

- The DataFrames `flights_df` and `airports_df` are available to you.

## Instructions

- Create a new DataFrame `normal_df` by joining `flights_df` with `airports_df`.
- Determine which type of join is used in the query plan.

In [1]:
# Intialization
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'

In [2]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext

In [ ]:
airports_df = spark.read.format("csv").option("header", "true").load("file://<pwd>/Dataset/airportnames.txt.gz")
flights_df = flights_df = spark.read.format('csv').option("header", "true").load('file://<pwd>/Dataset/AA_DFW_2018_Departures_Short.csv.gz')

spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1) # This is a critical call

# Join the flights_df and aiports_df DataFrames
normal_df = flights_df.____(____, \
    flights_df["Destination Airport"] == airports_df["IATA"] )

# Show the query plan
normal_df.____()

In [3]:
#departures_df = spark.read.csv('file:////home/khedkaramit20178016/hdp/pigandhive/devph/labs/pyspark/departures.txt.gz').distinct()
airports_df = spark.read.format("csv").option("header", "true").load("file:///home/talentum/test-jupyter/P3/M3/SM4/Dataset/airportnames.txt.gz")
flights_df = flights_df = spark.read.format('csv').option("header", "true").load('file:///home/talentum/test-jupyter/P3/M3/SM4/Dataset/AA_DFW_2018_Departures_Short.csv.gz')

spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1) # This is a critical call

# Join the flights_df and aiports_df DataFrames
normal_df = flights_df.join(airports_df, \
    flights_df["Destination Airport"] == airports_df["IATA"] )

# Show the query plan
normal_df.explain()

== Physical Plan ==
*(5) SortMergeJoin [Destination Airport#26], [IATA#11], Inner
:- *(2) Sort [Destination Airport#26 ASC NULLS FIRST], false, 0
:  +- Exchange hashpartitioning(Destination Airport#26, 200)
:     +- *(1) Project [Date (MM/DD/YYYY)#24, Flight Number#25, Destination Airport#26, Actual elapsed time (Minutes)#27]
:        +- *(1) Filter isnotnull(Destination Airport#26)
:           +- *(1) FileScan csv [Date (MM/DD/YYYY)#24,Flight Number#25,Destination Airport#26,Actual elapsed time (Minutes)#27] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/home/talentum/test-jupyter/P3/M3/SM4/Dataset/AA_DFW_2018_Departures_Short..., PartitionFilters: [], PushedFilters: [IsNotNull(Destination Airport)], ReadSchema: struct<Date (MM/DD/YYYY):string,Flight Number:string,Destination Airport:string,Actual elapsed ti...
+- *(4) Sort [IATA#11 ASC NULLS FIRST], false, 0
   +- Exchange hashpartitioning(IATA#11, 200)
      +- *(3) Project [AIRPORTNAME#10, IATA#11]
         +- *(3) 